In [81]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
import copy, math
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [15]:
dataset = pd.read_csv('/Users/huytuannguyen/Desktop/FPT/My self/MachineLearning/Logistic Regression/Exoplanet Hunting in Deep Space data/exoTrain.csv')
dataset.head()
dataset.shape

(5087, 3198)

In [14]:
df = dataset.dropna()
df

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,2,93.85,83.81,20.10,-26.98,-39.56,-124.71,-135.18,-96.27,-79.89,...,-78.07,-102.15,-102.15,25.13,48.57,92.54,39.32,61.42,5.08,-39.54
1,2,-38.88,-33.83,-58.54,-40.09,-79.31,-72.81,-86.55,-85.33,-83.97,...,-3.28,-32.21,-32.21,-24.89,-4.86,0.76,-11.70,6.46,16.00,19.93
2,2,532.64,535.92,513.73,496.92,456.45,466.00,464.50,486.39,436.56,...,-71.69,13.31,13.31,-29.89,-20.88,5.06,-11.80,-28.91,-70.02,-96.67
3,2,326.52,347.39,302.35,298.13,317.74,312.70,322.33,311.31,312.42,...,5.71,-3.73,-3.73,30.05,20.03,-12.67,-8.77,-17.31,-17.35,13.98
4,2,-1107.21,-1112.59,-1118.95,-1095.10,-1057.55,-1034.48,-998.34,-1022.71,-989.57,...,-594.37,-401.66,-401.66,-357.24,-443.76,-438.54,-399.71,-384.65,-411.79,-510.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5082,1,-91.91,-92.97,-78.76,-97.33,-68.00,-68.24,-75.48,-49.25,-30.92,...,139.95,147.26,156.95,155.64,156.36,151.75,-24.45,-17.00,3.23,19.28
5083,1,989.75,891.01,908.53,851.83,755.11,615.78,595.77,458.87,492.84,...,-26.50,-4.84,-76.30,-37.84,-153.83,-136.16,38.03,100.28,-45.64,35.58
5084,1,273.39,278.00,261.73,236.99,280.73,264.90,252.92,254.88,237.60,...,-26.82,-53.89,-48.71,30.99,15.96,-3.47,65.73,88.42,79.07,79.43
5085,1,3.82,2.09,-3.29,-2.88,1.66,-0.75,3.85,-0.03,3.28,...,10.86,-3.23,-5.10,-4.61,-9.82,-1.50,-4.65,-14.55,-6.41,-2.55


In [92]:
clone = df
clone = clone.drop(['LABEL'], axis=1)

clone

,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,FLUX.10,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,9.385000e-08,0.000838,0.000201,-0.000270,-0.000396,-0.001247,-0.001352,-9.627000e-04,-0.000799,-0.001602,...,-0.000781,-0.001022,-0.001022,0.000251,0.000486,0.000925,0.000393,0.000614,0.000051,-0.000395
1,-3.888000e-08,-0.000338,-0.000585,-0.000401,-0.000793,-0.000728,-0.000865,-8.533000e-04,-0.000840,-0.000734,...,-0.000033,-0.000322,-0.000322,-0.000249,-0.000049,0.000008,-0.000117,0.000065,0.000160,0.000199
2,5.326400e-07,0.005359,0.005137,0.004969,0.004564,0.004660,0.004645,4.863900e-03,0.004366,0.004844,...,-0.000717,0.000133,0.000133,-0.000299,-0.000209,0.000051,-0.000118,-0.000289,-0.000700,-0.000967
3,3.265200e-07,0.003474,0.003024,0.002981,0.003177,0.003127,0.003223,3.113100e-03,0.003124,0.003233,...,0.000057,-0.000037,-0.000037,0.000300,0.000200,-0.000127,-0.000088,-0.000173,-0.000174,0.000140
4,-1.107210e-06,-0.011126,-0.011189,-0.010951,-0.010575,-0.010345,-0.009983,-1.022710e-02,-0.009896,-0.009709,...,-0.005944,-0.004017,-0.004017,-0.003572,-0.004438,-0.004385,-0.003997,-0.003846,-0.004118,-0.005105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5082,-9.191000e-08,-0.000930,-0.000788,-0.000973,-0.000680,-0.000682,-0.000755,-4.925000e-04,-0.000309,-0.000119,...,0.001399,0.001473,0.001569,0.001556,0.001564,0.001517,-0.000244,-0.000170,0.000032,0.000193
5083,9.897500e-07,0.008910,0.009085,0.008518,0.007551,0.006158,0.005958,4.588700e-03,0.004928,0.003843,...,-0.000265,-0.000048,-0.000763,-0.000378,-0.001538,-0.001362,0.000380,0.001003,-0.000456,0.000356
5084,2.733900e-07,0.002780,0.002617,0.002370,0.002807,0.002649,0.002529,2.548800e-03,0.002376,0.002385,...,-0.000268,-0.000539,-0.000487,0.000310,0.000160,-0.000035,0.000657,0.000884,0.000791,0.000794
5085,3.820000e-09,0.000021,-0.000033,-0.000029,0.000017,-0.000008,0.000039,-3.000000e-07,0.000033,0.000063,...,0.000109,-0.000032,-0.000051,-0.000046,-0.000098,-0.000015,-0.000046,-0.000146,-0.000064,-0.000025


In [93]:
scaler = StandardScaler()
scaler.fit(df.drop('LABEL',axis=1))
data_scale = scaler.transform(df.drop('LABEL',axis=1))

In [99]:
data_scale.shape

tuple

array([-0.00235557, -0.00205404, -0.00579778, ...,  0.0341983 ,
        0.02736753,  0.01805157])

In [20]:
x_train = df.drop(['LABEL'], axis=1)
y_train = df['LABEL']
y_train.head()

0    2
1    2
2    2
3    2
4    2
Name: LABEL, dtype: int64

In [121]:
class LogisticRegressionModel():

    def __init__(self):
        self.weights = 0
        self.bias = 0
        self.cost_list = []
        self.epoch_list = []

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def cost(self, X, y, w, b, lambda_ =1 ):
        """
        Computes the cost over all examples
        Args:
        X : (ndarray Shape (m,n)) data, m examples by n features
        y : (array_like Shape (m,)) target value 
        w : (array_like Shape (n,)) Values of parameters of the model      
        b : scalar Values of bias parameter of the model
        lambda_: unused placeholder
        Returns:
        total_cost: (scalar)         cost 
        """

        m, n = X.shape
        total_cost = 0.0
        for i in range(m):
            z_i = np.dot(X[i],w) + b
            f_wb_i = self.sigmoid(z_i)
            total_cost +=  -y[i]*np.log(f_wb_i) - (1-y[i])*np.log(1-f_wb_i)
                
        total_cost = total_cost / m
        return total_cost
    
    def gradient(self, X, y, w, b):
        """
        Computes the gradient for linear regression 
    
        Args:
        X (ndarray (m,n): Data, m examples with n features
        y (ndarray (m,)): target values
        w (ndarray (n,)): model parameters  
        b (scalar)      : model parameter

        Returns
        dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w. 
        dj_db (scalar)      : The gradient of the cost w.r.t. the parameter b. 
        """

        # print(X)
        # print(y)

        m,n = X.shape
        dj_dw = np.zeros((n,))                           #(n,)
        dj_db = 0.
        print(dj_dw)

        for i in range(m):
            f_wb_i = self.sigmoid(np.dot(X[i],w) + b)          #(n,)(n,)=scalar
            err_i  = f_wb_i  - y[i]                       #scalar
            dj_db = dj_db + err_i
            
            for j in range(n):
                # dj_dw[j] = dj_dw[j] + err_i * X[i,j]      #scalar
                ttt = err_i * X[i][j]
                print(dj_dw[j] + err_i * X[i][j])
                dj_dw[j] = dj_dw[j] + err_i * X[i][j]


        dj_dw = dj_dw/m                                   #(n,)
        dj_db = dj_db/m      
        return dj_db, dj_dw
    
    def gradient_descent(self,X, y, beta, learning_rate):
        y = y.reshape(-1, 1)
        gradients = np.dot(X.T, self.sigmoid(np.dot(X, beta.T)) - y) / len(y)
        new_betas = beta - learning_rate * gradients.T

        return new_betas
    
    def train(self, X, y, epochs, batch_size, learning_rate = 0.01):
        """
        Performs batch gradient descent
        
        Args:
        X (ndarray (m,n)   : Data, m examples with n features
        y (ndarray (m,))   : target values
        alpha (float)      : Learning rate
        epochs (scalar) : number of iterations to run gradient descent
        
        Returns:
        w (ndarray (n,))   : Updated values of parameters
        b (scalar)         : Updated value of parameter 
        """

        # init parameters
        self.weights = np.zeros_like(X.shape[0])  #avoid modifying global w within function
        self.bias = 0
        total_samples = X.shape[0]
        
        if batch_size > total_samples: # In this case mini batch becomes same as batch gradient descent
            batch_size = total_samples
        
        num_batches = int(total_samples/batch_size)

        for i in range(epochs):
            random_indices = np.random.permutation(total_samples)
            # print(type(random_indices))
            # print(type(X))
            X_tmp = X[random_indices,:]
            y_tmp = y[random_indices]
            

            for j in range(0,total_samples,batch_size):
                Xj = X_tmp[j:j+batch_size]
                yj = y_tmp[j:j+batch_size]

                y_pred = self.sigmoid(Xj)
                # Calculate the gradient and update the parameters
                dj_db, dj_dw = self.gradient(X, y, self.weights, self.bias)   

                # Update Parameters using w, b, alpha and gradient
                self.weights = self.weights - learning_rate * dj_dw               
                self.bias = self.bias - learning_rate * dj_db            

                cost = self.cost(Xj, yj, self.weights, self.bias)   
                # cost_list.append(cost)

            if i%10 == 0:
                self.cost_list.append(cost)
                self.epoch_list.append(i)
        
            
        # return w, b, cost_list, epoch_list        #return final w,b and J history for graphing

    def predict(self, X, w, b):
        """
        Predict whether the label is 0 or 1 using learned logistic
        regression parameters w
        
        Args:
        X : (ndarray Shape (m, n))
        w : (array_like Shape (n,))      Parameters of the model
        b : (scalar, float)              Parameter of the model

        Returns:
        p: (ndarray (m,1))
            The predictions for X using a threshold at 0.5
        """
        # number of training examples
        m, n = X.shape   
        p = np.zeros(m)
    
        # Loop over each example
        for i in range(m):   

            # Calculate f_wb (exactly how you did it in the compute_cost function above) 
            # using a couple of lines of code
            f_wb = self.sigmoid(np.dot(X[i],w) + b)

            # Calculate the prediction for that training example 
            if f_wb >= 0.5:
                p[i] = 1
            else:
                p[i] = 0
    
        return p

In [76]:
print(x_train.shape, type(x_train))
print(y_train.shape, type(y_train))

(5087, 3197) <class 'pandas.core.frame.DataFrame'>
(5087,) <class 'pandas.core.series.Series'>


In [80]:
ttt = x_train.astype('float128').dtypes
ttt

FLUX.1       float128
FLUX.2       float128
FLUX.3       float128
FLUX.4       float128
FLUX.5       float128
               ...   
FLUX.3193    float128
FLUX.3194    float128
FLUX.3195    float128
FLUX.3196    float128
FLUX.3197    float128
Length: 3197, dtype: object

In [122]:
model = LogisticRegressionModel()
model.train(data_scale, y_train, 120, 100)

[0. 0. 0. ... 0. 0. 0.]
[0.00353335 0.00353335 0.00353335 ... 0.00353335 0.00353335 0.00353335]


ValueError: setting an array element with a sequence.

In [67]:
model_2 = LogisticRegression(random_state=0).fit(x_train, y_train)

/Users/huytuannguyen/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [68]:
testset = pd.read_csv('/Users/huytuannguyen/Desktop/FPT/My self/MachineLearning/Logistic Regression/Exoplanet Hunting in Deep Space data/exoTrain.csv')
dataset.shape
x_test = testset.drop(['LABEL'], axis=1)
y_test = testset['LABEL']


In [69]:
model_2.predict(x_test)

array([1, 1, 2, ..., 1, 2, 1])

In [74]:
model_2.score(x_test, y_test.to_numpy())

0.6587379595046197